# 推薦システムとimplicitライブラリの利用方法

推薦システムは、ユーザーの嗜好や行動を分析し、個別に最適なアイテムを推薦するシステムである。その中で、Pythonのライブラリであるimplicitは特に有名であり、効率的な計算と使いやすさが特徴である。ここでは、implicitライブラリの使い方と、movielens-100kデータセットを利用した具体例について説明する。

## ソースコード

この記事で利用するソースコードは以下の通りである。


### github
- jupyter notebook形式のファイルは[こちら](https://github.com/hiroshi0530/wa-src/blob/master/rec/rec/01/01_nb.ipynb)

### google colaboratory
- google colaboratory で実行する場合は[こちら](https://colab.research.google.com/github/hiroshi0530/wa-src/blob/master/rec/rec/01/01_nb.ipynb)


## 実行環境
OSはmacOSである。LinuxやUnixのコマンドとはオプションが異なりますので注意していただきたい。

In [1]:
!sw_vers

ProductName:		macOS
ProductVersion:		13.5.1
BuildVersion:		22G90


In [2]:
!python -V

Python 3.9.17


pandasのテーブルを見やすいようにHTMLのテーブルにCSSの設定を行う。

In [3]:
from IPython.core.display import HTML

style = """
<style>
    .dataframe thead tr:only-child th {
        text-align: right;
    }

    .dataframe thead th {
        text-align: left;
        padding: 5px;
    }

    .dataframe tbody tr th {
        vertical-align: top;
        padding: 5px;
    }

    .dataframe tbody tr:hover {
        background-color: #ffff99;
    }

    .dataframe {
        background-color: white;
        color: black;
        font-size: 16px;
    }

</style>
"""
HTML(style)

基本的なライブラリをインポートし watermark を利用してそのバージョンを確認する。
ついでに乱数のseedの設定する。

In [4]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [5]:
import random
import numpy as np
import pandas as pd

import implicit

seed = 123
random_state = 123

random.seed(seed)
np.random.seed(seed)

from watermark import watermark

print(watermark(python=True, watermark=True, iversions=True, globals_=globals()))

Python implementation: CPython
Python version       : 3.9.17
IPython version      : 8.17.2

pandas  : 2.0.3
numpy   : 1.25.2
implicit: 0.7.0

Watermark: 2.4.3



/Users/hiroshi/anaconda3/envs/20240101_blog/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 推薦システムで利用されるimplicitの実装例

<!--
## 目次
1. 推薦システムとは
2. implicitライブラリの概要
3. Movielens-100kデータセットの概要
4. ALSアルゴリズムの詳細と実装
5. BPRアルゴリズムの詳細と実装
6. 実装例
7. メリットとデメリット
8. 結論
-->

## 推薦システムとは

推薦システムとは、ユーザーの嗜好に基づいてアイテムを推薦するシステムである。例えば、Netflixではユーザーが視聴した映画に基づいて新しい映画を推薦する。推薦システムの種類には、大きく分けて協調フィルタリングとコンテンツベースフィルタリングがある。

### 協調フィルタリング

協調フィルタリングは、ユーザーの過去の行動や評価に基づいてアイテムを推薦する手法である。具体的には、ユーザー行動行列を用いる。

### コンテンツベースフィルタリング

コンテンツベースフィルタリングは、アイテムの特徴や属性に基づいてアイテムを推薦する手法である。

## implicitライブラリの概要

implicitはPythonで書かれたライブラリであり、特に協調フィルタリングのアルゴリズムを実装するために用いられる。implicitは主に以下のアルゴリズムをサポートしている。

- ALS（Alternating Least Squares）
- BPR（Bayesian Personalized Ranking）

## Movielens-100kデータセットの概要

Movielens-100kは映画の評価データセットであり、100,000件の評価データが含まれている。このデータセットを用いることで、推薦システムの性能を評価することができる。

## ALSアルゴリズムの詳細と実装

ALS（交互最小二乗法）は、ユーザーとアイテムの行列を因子分解する手法である。ALSでは、ユーザー行列とアイテム行列を交互に更新することで、予測行列を近似する。

### ALSの表式

ALSの基本的な考え方は、ユーザー行列$\mathbf{U}$とアイテム行列$\mathbf{I}$を求めることである。評価行列$\mathbf{R}$は次のように近似される。

$$
\mathbf{R} \approx \mathbf{U} \mathbf{I}^T
$$

ここで、ALSは次の最小化問題を解く。

$$
\min_{\mathbf{U}, \mathbf{I}} \|\| \mathbf{R} - \mathbf{U} \mathbf{I}^T \|\|^2_F + \lambda ( \|\| \mathbf{U} \|\|^2_F + \|\| \mathbf{I} \|\|^2_F )
$$

ここで、$\|\| \cdot \|\|_F$はフロベニウスノルムを表し、$\lambda$は正則化パラメータである。

### ALSの実装

次に、implicitライブラリを用いたALSの実装例を示す。


In [6]:
import implicit
from scipy.sparse import coo_matrix
from pprint import pprint

# データの読み込みと前処理
df = pd.read_csv("./ml-100k/u.data", sep="\t", names=["user_id", "item_id", "rating", "timestamp"])
rows = df["user_id"].astype(int)
cols = df["item_id"].astype(int)
values = df["rating"].astype(float)

df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [7]:
# 評価行列を作成
R = coo_matrix((values, (rows, cols)))

# coo_matrixをcsr_matrixに変換
R = R.tocsr()

# ALSモデルの訓練
model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=50)
model.fit(R)

# ユーザーとアイテムの行列
U = model.user_factors
I = model.item_factors

# 結果の表示
pprint(U.round(2))
pprint(I.round(2))

100%|██████████████████████████████████████████| 50/50 [00:01<00:00, 42.50it/s]

array([[ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
       [ 0.92,  0.85,  0.25, ..., -0.11,  1.78,  0.34],
       [ 0.59,  0.67,  0.08, ...,  0.96, -0.71,  0.13],
       ...,
       [ 0.73,  0.18,  0.05, ..., -0.21, -0.54,  0.03],
       [ 1.79,  0.88, -0.73, ..., -0.11,  0.6 ,  0.36],
       [ 0.65,  0.87, -0.36, ..., -0.7 , -0.49,  0.39]], dtype=float32)
array([[ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
       [ 0.21,  0.07, -0.14, ..., -0.07, -0.18,  0.02],
       [ 0.07,  0.06, -0.01, ..., -0.12, -0.06,  0.03],
       ...,
       [ 0.  ,  0.01,  0.  , ...,  0.  ,  0.  , -0.  ],
       [ 0.01,  0.01, -0.01, ..., -0.  , -0.01, -0.01],
       [ 0.  , -0.  ,  0.01, ...,  0.  ,  0.01, -0.  ]], dtype=float32)


## BPRアルゴリズムの詳細と実装

BPR（ベイジアン個人化ランキング）は、ランキングを最適化するための手法である。BPRは、ユーザーのペアワイズな選好を最大化することを目的とする。

### BPRの数式

BPRは、ユーザーがあるアイテムを他のアイテムよりも好む確率を最大化する。具体的には、次の対数尤度関数を最大化する。

$$
\sum_{(u,i,j) \in D} \ln \sigma (\hat{x}\_{u,i} - \hat{x}\_{u,j}) - \lambda \|\| \Theta \|\|^2
$$

ここで、$\sigma$はシグモイド関数、$\hat{x}_{u,i}$はユーザー$u$がアイテム$i$に対して持つスコア、$D$はデータセット、$\Theta$はモデルパラメータである。

### BPRの実装

次に、implicitライブラリを用いたBPRの実装例を示す。

In [8]:
import implicit
from scipy.sparse import coo_matrix
from pprint import pprint

# データの読み込みと前処理
df = pd.read_csv("./ml-100k/u.data", sep="\t", names=["user_id", "item_id", "rating", "timestamp"])
rows = df["user_id"].astype(int)
cols = df["item_id"].astype(int)
values = df["rating"].astype(float)

df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [9]:
# 評価行列を作成
R = coo_matrix((values, (rows, cols)))

# coo_matrixをcsr_matrixに変換
R = R.tocsr()

# BPRモデルの訓練
model = implicit.bpr.BayesianPersonalizedRanking(factors=20, regularization=0.1, iterations=50)
model.fit(R)

# ユーザーとアイテムの行列
U = model.user_factors
I = model.item_factors

# 結果の表示
pprint(U)
pprint(I)

100%|████████| 50/50 [00:00<00:00, 97.29it/s, train_auc=76.96%, skipped=30.50%]

array([[ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  1.0000000e+00],
       [-2.9951489e-02, -3.1067370e-02, -7.8605428e-02, ...,
        -5.1104784e-02,  4.5966309e-02,  1.0000000e+00],
       [ 8.8656984e-02,  8.4776938e-02,  2.8001857e-01, ...,
         1.1082027e-01, -1.1200863e-01,  1.0000000e+00],
       ...,
       [ 9.1114938e-03,  1.5749291e-02,  2.3070066e-01, ...,
         5.2926857e-02, -8.5393496e-02,  1.0000000e+00],
       [ 7.1497530e-02,  4.6532072e-02, -1.7651236e-01, ...,
         1.8227158e-02,  3.4350850e-02,  1.0000000e+00],
       [-1.3805275e-01, -1.0069024e-01,  4.7426656e-02, ...,
        -6.7706250e-02,  5.8717502e-04,  1.0000000e+00]], dtype=float32)
array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.03371504, -0.03552511,  0.15254132, ...,  0.00072459,
        -0.0323111 ,  0.6411407 ],
       [-0.11410373, -0.07173929,  0.04195825, ..., -0.04867549,

## 実装例

具体的な実装例として、movielens-100kデータセットを用いて、ALSおよびBPRモデルを構築する。以下にその手順を示す。

### データの準備

まず、データを読み込み、前処理を行う。

In [10]:
import implicit
import pandas as pd
import numpy as np

from scipy.sparse import coo_matrix
from sklearn.model_selection import train_test_split

# データの読み込みと前処理
df = pd.read_csv("./ml-100k/u.data", sep="\t", names=["user_id", "item_id", "rating", "timestamp"])

# トレーニングとテストデータに分割
train, test = train_test_split(df, test_size=0.2)

# 評価行列を作成
train_matrix = coo_matrix((train["rating"], (train["user_id"], train["item_id"])))
test_matrix = coo_matrix((test["rating"], (test["user_id"], test["item_id"])))

# coo_matrixをcsr_matrixに変換
train_matrix = train_matrix.tocsr()
test_matrix = test_matrix.tocsr()

### ALSモデルの訓練と評価

次に、ALSモデルを訓練し、評価する。

In [11]:
# ALSモデルの訓練
als_model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=50)
als_model.fit(train_matrix)

# テストデータに対する予測
test_predictions = als_model.recommend_all(test_matrix)


def compute_precision(true_matrix, pred_matrix, k=10):
    """
    精度を計算する関数

    Parameters:
    - true_matrix (coo_matrix): 実際の評価行列
    - pred_matrix (ndarray): 予測された評価行列
    - k (int): 精度を計算する際のトップKアイテムの数

    Returns:
    - precision (float): 精度
    """
    # 実際の評価行列をリストに変換
    true_items = true_matrix.tolil().rows

    # 予測されたアイテムのインデックスを取得
    pred_items = np.argsort(-pred_matrix, axis=1)[:, :k]

    # ユーザーごとの精度を計算
    precisions = []
    for user_id in range(len(true_items)):
        true_set = set(true_items[user_id])
        pred_set = set(pred_items[user_id])

        if len(true_set) > 0:
            precision = len(true_set & pred_set) / min(len(true_set), k)
            precisions.append(precision)

    # 平均精度を計算
    return np.mean(precisions)


# 使用例
true_matrix = test_matrix  # テストデータの実際の評価行列
pred_matrix = als_model.recommend_all(test_matrix)  # ALSモデルによる予測

precision = compute_precision(true_matrix, pred_matrix)
print(f"ALSモデルの精度: {precision:.3f}")

100%|██████████████████████████████████████████| 50/50 [00:01<00:00, 46.76it/s]


IndexError: index 1682 is out of bounds for axis 1 with size 1682

### BPRモデルの訓練と評価

同様に、BPRモデルを訓練し、評価する。

In [ ]:
# BPRモデルの訓練
bpr_model = implicit.bpr.BayesianPersonalizedRanking(factors=20, regularization=0.1, iterations=50)
bpr_model.fit(train_matrix)

# テストデータに対する予測
test_predictions = bpr_model.recommend_all(test_matrix)

# 精度の評価
precision = compute_precision(test_matrix, test_predictions)
print(f"BPRモデルの精度: {precision:.3f}")

## メリットとデメリット

### ALSのメリットとデメリット

ALSのメリットは、スパースなデータにも対応できることである。また、並列化が容易であり、大規模データセットにも適用可能である。デメリットは、行列の因子分解に時間がかかることがある。

### BPRのメリットとデメリット

BPRのメリットは、ランキングに特化しているため、推薦の質が高いことである。デメリットは、

訓練に時間がかかることがある。

## 結論

この記事では、implicitライブラリを用いてALSおよびBPRアルゴリズムを実装し、movielens-100kデータセットでの具体例を紹介した。これにより、推薦システムの構築方法を理解できる。また、各アルゴリズムのメリットとデメリットについても解説した。

### 参考文献

- "Collaborative Filtering for Implicit Feedback Datasets", Hu, Y., Koren, Y., and Volinsky, C., 2008.
- "BPR: Bayesian Personalized Ranking from Implicit Feedback", Rendle, S., Freudenthaler, C., Gantner, Z., and Schmidt-Thieme, L., 2009.
- Movielens Dataset: https://grouplens.org/datasets/movielens/100k/